1. Clone CLEAN (v1.0.1) from GitHub

In [1]:
!git clone https://github.com/tttianhao/CLEAN.git

Cloning into 'CLEAN'...
remote: Enumerating objects: 954, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 954 (delta 96), reused 87 (delta 82), pack-reused 826
Receiving objects: 100% (954/954), 449.22 MiB | 15.31 MiB/s, done.
Resolving deltas: 100% (524/524), done.
Updating files: 100% (39/39), done.


2. In Colab, change Directory to /content/CLEAN/app
Then you could see:
['gmm.py',
 'train-supconH.py',
 'requirements.txt',
 'results',
 'build.py',
 'data',
 'DEMO.ipynb',
 'CLEAN_infer_fasta.py',
 'src',
 'inference.py',
 'train-triplet.py']

In [2]:
import os
os.chdir('/content/CLEAN/app')
current_path = os.getcwd()
os.listdir(current_path)

['requirements.txt',
 'CLEAN_infer_fasta.py',
 'src',
 'data',
 'DEMO.ipynb',
 'gmm.py',
 'inference.py',
 'train-supconH.py',
 'train-triplet.py',
 'build.py',
 'results']

3. Install the latest release of esm, clone esm gits from GitHub

In [3]:
!pip install fair-esm  # latest release
!git clone https://github.com/facebookresearch/esm.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 1.2 MB/s eta 0:00:00
Cloning into 'esm'...
remote: Enumerating objects: 1511, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 1511 (delta 46), reused 102 (delta 36), pack-reused 1360
Receiving objects: 100% (1511/1511), 11.26 MiB | 18.57 MiB/s, done.
Resolving deltas: 100% (900/900), done.


4. Retrive esm from dataset, upload your dataset to ./data

In [4]:
file_path = 'src/CLEAN/utils.py'
with open(file_path, 'r') as file:
    content = file.read()
content = content.replace('esm1b_t33_650M_UR50S', 'esm2_t6_8M_UR50D')

In [6]:
from src.CLEAN.utils import csv_to_fasta, mutate_single_seq_ECs, retrive_esm1b_embedding, compute_esm_distance
csv_to_fasta("data/combined_columns.csv", "data/combined_columns.fasta")
retrive_esm1b_embedding("combined_columns")

5. Mutate single sequences and retrive esm from the mutated sequences

In [7]:
# from src.CLEAN.utils import mutate_single_seq_ECs, retrive_esm1b_embedding
# train_file = "combined_columns"
# train_fasta_file = mutate_single_seq_ECs(train_file)
# train_fasta_file

In [8]:
# retrive_esm1b_embedding(train_fasta_file)

6. Compute default esm distance for training

In [9]:
# from src.CLEAN.utils import compute_esm_distance
train_file = "combined_columns"
compute_esm_distance(train_file)

100%|██████████| 9/9 [00:00<00:00, 732.71it/s]


Calculating distance map, number of unique EC is 9


9it [00:00, 129.99it/s]


7. Triplet loss training

In [16]:
file_path = 'train-triplet.py'
with open(file_path, 'r') as file:
    content = file.read()
content = content.replace('from CLEAN.', 'from src.CLEAN.')
with open(file_path, 'w') as file:
    file.write(content)
with open('data/model/combined_columns.pth', 'w') as file:
    pass
!python train-triplet.py --training_data combined_columns --model_name combined_columns --epoch 2500

Streaming output truncated to the last 5000 lines.
---------------------------------------------------------------------------
| end of epoch 852 | time:  0.01s | training loss 0.1369
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch 853 | time:  0.01s | training loss 0.1833
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch 854 | time:  0.01s | training loss 0.3498
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch 855 | time:  0.01s | training loss 0.0000
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch 856 | time:  0.01s | t

8. Supcon loss training

In [20]:
file_path = 'train-supconH.py'
with open(file_path, 'r') as file:
    content = file.read()
content = content.replace('from CLEAN.', 'from src.CLEAN.')
with open(file_path, 'w') as file:
    file.write(content)
with open('data/model/combined_columns.pth', 'w') as file:
    pass
!python train-supconH.py --training_data combined_columns --model_name combined_columns --epoch 1500 --n_pos 9 --n_neg 30 -T 0.1

==> device used: cuda:0 | dtype used:  torch.float32 
==> args: Namespace(learning_rate=0.0005, epoch=1500, model_name='combined_columns', training_data='combined_columns', temp=0.1, n_pos=9, n_neg=30, hidden_dim=512, out_dim=256, adaptive_rate=60, verbose=False)
The number of unique EC numbers:  9
---------------------------------------------------------------------------
| end of epoch   1 | time:  0.90s | training loss 3.7207
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   2 | time:  0.13s | training loss 3.6523
---------------------------------------------------------------------------
---------------------------------------------------------------------------
| end of epoch   3 | time:  0.14s | training loss 3.6775
---------------------------------------------------------------------------
-------------------------------------------------------------------------